In [1]:
import pandas as pd
import numpy as np
import requests

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url)

In [3]:
len(response.content)

79071

In [4]:
df = pd.read_html(response.text)[0] #Read the first table contained in response.text html file.
df.columns = ['Postal Code', 'Borough', 'Neighborhood'] # Name the Data Frame columns.
df.drop([0], axis=0, inplace=True) # Get rid of first column.
df.reset_index(inplace=True, drop=True) # Re set the index without creating a new columns called "index"
df.head()

,Postal Code,Borough,Neighborhood
0,M2A,Not assigned,Not assigned
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,Harbourfront
4,M5A,Downtown Toronto,Regent Park


In [5]:
df.shape

(287, 3)

In [6]:

useless_rows = df[df['Borough'] == 'Not assigned']
print('Useless rows: {}'.format(useless_rows.shape[0]))
useless_rows.head()

Useless rows: 76


,Postal Code,Borough,Neighborhood
0,M2A,Not assigned,Not assigned
8,M8A,Not assigned,Not assigned
12,M2B,Not assigned,Not assigned
19,M7B,Not assigned,Not assigned
20,M8B,Not assigned,Not assigned


In [7]:
df = df[df['Borough'] != 'Not assigned']
print('Useful rows: {}'.format(df.shape[0]))
df.head()

Useful rows: 211


,Postal Code,Borough,Neighborhood
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,Harbourfront
4,M5A,Downtown Toronto,Regent Park
5,M6A,North York,Lawrence Heights


In [8]:
index_neighs_to_replace_for_boroughs = df[(df['Borough'] != 'Not assigned') & (df['Neighborhood'] == 'Not assigned')].index.tolist()

for i in index_neighs_to_replace_for_boroughs:
    print('Neighborhood with this value: "'+ df['Neighborhood'][i] + '" will be replaced with this: "' + df['Borough'][i] + '" value')
    df['Neighborhood'][i] = df['Borough'][i]

Neighborhood with this value: "Not assigned" will be replaced with this: "Queen's Park" value


In [9]:
if all((Borough, Neighbourhood != 'Not assigned') for i, Borough, Neighbourhood in df[['Borough','Neighborhood']].itertuples()):
    print('All Borough and Neighbourhood have assigned values.')

All Borough and Neighbourhood have assigned values.


In [10]:
df.shape


(211, 3)

In [11]:
df.head()

,Postal Code,Borough,Neighborhood
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,Harbourfront
4,M5A,Downtown Toronto,Regent Park
5,M6A,North York,Lawrence Heights


In [12]:
df = df.astype(str).set_index(['Postal Code', 'Borough'])
df_merged = df.groupby(level=['Postal Code', 'Borough'], sort=False).agg(', '.join)
df_merged.head()

,,Neighborhood
Postal Code,Borough,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Harbourfront, Regent Park"
M6A,North York,"Lawrence Heights, Lawrence Manor"
M7A,Queen's Park,Queen's Park


In [13]:
df_merged.reset_index(inplace=True)
df_merged.head(12)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [14]:
df_latlong = pd.read_csv('http://cocl.us/Geospatial_data')
df_latlong.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
df_latlong.sort_values(by='Postal Code').head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
df_merged.sort_values(by='Postal Code', inplace=True)
df_merged.reset_index(inplace=True, drop=True)
df_merged.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [17]:
df_places = pd.concat([df_merged, df_latlong[['Latitude', 'Longitude']]], axis=1)

In [18]:
df_places.head(12)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
